In [85]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ast import literal_eval
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
import sympy as sym
from sympy.parsing import mathematica

In [86]:

RESULTS_DIR = "final_results"
benchmarks = [
    "rational/plain",
    "rational_2/plain",
    "sqrt/plain",
    "rational_sqrt/plain",

    # "rational/with_constant",
]

In [87]:
def process_output(row):
    if pd.isnull(row):
        return (None, None)

    try:
        row = float(row)
        return (row, row)
    except:
        pass

    try:
        return literal_eval(str(row).strip())
    except Exception as e:
        pass
    
    try:
        for w in ['integrate', 'boole']:
            if w in row.lower():
                return (None, None)        
        
        row = row.replace("[",'(').replace("]", ")").replace("L", "l").replace("E(p_) =", "").replace("^", "**").strip()
        row = sym.parse_expr(row).evalf()
        
        return (row,row)
        
    except Exception as e:
        print(e, row)
    
        
    return (None, None)

In [88]:
def process_time(row):
    if pd.isnull(row):
        return None

    try:
        row = float(row)
        return row
    except:
        pass

    try:
        return literal_eval(str(row).strip())[0]
    except Exception as e:
        pass
        
    except Exception as e:
        print(e, row)
    
        
    return None

In [89]:
def process_benchmark(benchmark):
    df_list = []
    for f_name in os.listdir(os.path.join(RESULTS_DIR, benchmark)):
        tool_name =  f_name.split("_")[-2]
        df_ = pd.read_csv(os.path.join(RESULTS_DIR, benchmark, f_name))
        df_['tool'] = tool_name
        df_list.append(df_[['index', 'tool','output', 'time']])
    df = pd.concat(df_list)

    df['output'] = df['output'].apply(process_output)
    df['time'] = df['time'].apply(process_time)
    
    df['lower'], df['upper'] = df['output'].apply(lambda o: o[0]),df['output'].apply(lambda o: o[1])
    df['error'] = df['upper']-df['lower']
    df['benchmark'] = benchmark
    df['is_solved'] = (~pd.isnull(df['error']))&(df['time']<=3600)
    return df

In [90]:
df_b_list = []
for b in benchmarks:
    df_b_list.append(process_benchmark(b))

df_b = pd.concat(df_b_list)

In [91]:
df_b[(df_b.tool=='psi' )& (df_b.benchmark=='rational_2/plain')]

,index,tool,output,time,lower,upper,error,benchmark,is_solved


In [92]:
prop = 'error'
method = 'std'

df_ = df_b[[prop, 'tool', 'benchmark']].groupby(['benchmark','tool']).agg({prop:[method]}).reset_index()
df_.columns = df_.columns.droplevel(1)
df_ = df_.pivot_table(columns='tool', index='benchmark', values=prop)
df_

tool,faza,gubpi,latte,psi,volesti
benchmark,,,,,
rational/plain,0.015133,2.130043,0.0,0.0,0.295519
rational_2/plain,0.000073,0.430110,NaN,NaN,0.098615
rational_sqrt/plain,0.040102,0.000814,NaN,NaN,NaN
sqrt/plain,0.000364,0.455869,NaN,0.0,NaN


In [93]:
df_.to_csv(os.path.join(RESULTS_DIR, f'{prop}_{method}.csv'))